In [1]:
!pip install -qq transformers tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.1 MB/s eta 0:00:00


In [3]:
with open("all-messages.txt", "r", encoding="utf8") as f:
    messages = f.readlines()
    messages = [x.strip() for x in messages]

To remove non-alphabetic characters like emojis and special characters, we can use regular expressions. Here's a sample code that removes all non-alphabetic characters and special characters:

In [4]:
import re

# regular expression pattern to remove non-alphabetic characters and special characters
pattern = r'[^a-zA-Z\s\[\]]'

# apply pattern on each message and add cleaned message to a new list
cleaned_messages = []
for message in messages:
    cleaned_message = re.sub(pattern, '', message)
    cleaned_messages.append(cleaned_message)

print("Number of messages after cleaning:", len(cleaned_messages))

Number of messages after cleaning: 127309


In [5]:
cleaned_messages

['[others] ',
 '[others] Volevo fa na prova',
 '[others] Se cambia il nome utente',
 '[others] sono sempre io',
 '[others] maurizio valitutto',
 '[others] Eh a me mi hanno cacciato',
 '[others] non puoi rientrare',
 '[others] Il fatto  che non capisco se se ne accorgono percio volevo fare dei test',
 '[others] pure a me mi hanno rimosso ahah',
 '[others] Hahah ma cosi senza motivo',
 '[others] si',
 '[others] Allora pensano che sia sempre io',
 '[others] Hahahaha',
 '[others] ma cmq si vede lo username che penso sia fisso',
 '[others] Nono cambia',
 '[others] Si puo cambiare',
 '[others] Solo che praticamente quando cacci',
 '[others] Tipo esce scritto tom cruise ha lasciato il grupp',
 '[others] Quando rietro con un altro nome',
 '[others] E me cambia quello vecchio dove appariva tom cruise',
 '[others] Ed  esce il nuovo',
 '[others] In tempo reale',
 '[others] Vorrei capire se cambia solo a me o a tutti',
 '[others] Pero se hanno bannato anche te vuol dire che stanno bannando tutti',

Great! The next step is to clone Lorenzo's chat as the training dataset. We'll create a new list called lorenzo_chat that contains all the messages in which Lorenzo participated. 

In [6]:
# replace "Lorenzo Valitutto" with the name of the person you want to clone
lorenzo_chat = []
for message in cleaned_messages:
    if "[me]" in message:
        lorenzo_chat.append(message)

print("Number of messages in Lorenzo's chat:", len(lorenzo_chat))

Number of messages in Lorenzo's chat: 9649


Great! Let's move on to the next step which is to split the messages to make it more like a conversation.

For this, we need to create a list of conversations where each conversation is a list of messages. Each conversation should start with a message written by Lorenzo, followed by a message written by another participant, and so on.

In [7]:
# create a list of conversations
conversations = []

# initialize conversation with the first message
conversation = [cleaned_messages[0]]

# add messages to the conversation
for i in range(1, len(cleaned_messages)):
    # check if current message was written by Lorenzo
    if "[me]" in cleaned_messages[i]:
        # add current conversation to list of conversations
        conversations.append(conversation)
        # start new conversation with the message written by Lorenzo
        conversation = [cleaned_messages[i]]
    else:
        # add message to the current conversation
        conversation.append(cleaned_messages[i])

# add last conversation to list of conversations
conversations.append(conversation)

print("Number of conversations:", len(conversations))

Number of conversations: 9650


In [8]:
conversations[0]

['[others] ',
 '[others] Volevo fa na prova',
 '[others] Se cambia il nome utente',
 '[others] sono sempre io',
 '[others] maurizio valitutto',
 '[others] Eh a me mi hanno cacciato',
 '[others] non puoi rientrare',
 '[others] Il fatto  che non capisco se se ne accorgono percio volevo fare dei test',
 '[others] pure a me mi hanno rimosso ahah',
 '[others] Hahah ma cosi senza motivo',
 '[others] si',
 '[others] Allora pensano che sia sempre io',
 '[others] Hahahaha',
 '[others] ma cmq si vede lo username che penso sia fisso',
 '[others] Nono cambia',
 '[others] Si puo cambiare',
 '[others] Solo che praticamente quando cacci',
 '[others] Tipo esce scritto tom cruise ha lasciato il grupp',
 '[others] Quando rietro con un altro nome',
 '[others] E me cambia quello vecchio dove appariva tom cruise',
 '[others] Ed  esce il nuovo',
 '[others] In tempo reale',
 '[others] Vorrei capire se cambia solo a me o a tutti',
 '[others] Pero se hanno bannato anche te vuol dire che stanno bannando tutti',

In [9]:
def trim_conversations(conversations, max_len=10):
    """
    Trims a list of conversations to a maximum length of `max_len` elements.
    """
    trimmed_conversations = []
    for conversation in conversations:
        if len(conversation) > max_len:
            # trim the conversation to `max_len` elements
            conversation = conversation[:max_len]
        trimmed_conversations.append(conversation)
    return trimmed_conversations

trimmed_conversations = trim_conversations(conversations)


In [10]:
trimmed_conversations[714]

['[me] a domani',
 '[others] Va bene',
 '[others] A domani',
 '[others] Buona serata',
 '[others] Bellissima Lorenzo come lhai chiamata',
 '[others]  una biscia',
 '[others] Ok a domani',
 '[others] Sono Colin Canon',
 '[others] Lmao',
 '[others] Hahahahahahahaaha']

In [11]:
with open("output.txt", "w") as f:
    for conversation in trimmed_conversations:
        for message in conversation:
            f.write(message.replace("[me] ","").replace("[others] ","") + "\n")

Great! The next step is to fine-tune a GPT language model using the conversations data.

For this, we'll use the GPT2LMHeadModel class from the transformers library. We'll use the base version of GPT-2 with 117M parameters. We'll also use the LineByLineTextDataset class from the transformers library to preprocess the input data.

In [14]:
from transformers import BertTokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, GPT2LMHeadModel

# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-italian-uncased')

# initialize text dataset
text_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="output.txt",  # replace with the path to your Italian text
    block_size=128
)

# initialize data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# initialize training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    save_steps=1000,
    logging_steps=1000,
    max_steps=1000
)

# initialize GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2', output_hidden_states=True)

# initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=text_dataset,
    data_collator=data_collator
)

# train the model
trainer.train()

Step,Training Loss
1000,6.691600


TrainOutput(global_step=1000, training_loss=6.69158154296875, metrics={'train_runtime': 549.4448, 'train_samples_per_second': 29.12, 'train_steps_per_second': 1.82, 'total_flos': 1040922132480000.0, 'train_loss': 6.69158154296875, 'epoch': 5.56})

To save the trained model and tokenizer files for later use, you can use the save_pretrained() method of the GPT2LMHeadModel and GPT2Tokenizer classes. This method saves the model weights and configuration to disk, along with the tokenizer files.

In [92]:
# mount Google Drive to the Colab environment
from google.colab import drive
drive.mount('/content/gdrive')

# set the path to save the model and tokenizer files
model_save_path = "/content/gdrive/MyDrive/model"
tokenizer_save_path = "/content/gdrive/MyDrive/tokenizer"

# save the model and tokenizer files
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

Mounted at /content/gdrive


('/content/gdrive/MyDrive/tokenizer/tokenizer_config.json',
 '/content/gdrive/MyDrive/tokenizer/special_tokens_map.json',
 '/content/gdrive/MyDrive/tokenizer/vocab.txt',
 '/content/gdrive/MyDrive/tokenizer/added_tokens.json')

Great! Now that you've fine-tuned the GPT-2 model on the preprocessed and split conversations data, you can use it to generate new messages that look like they were written by Lorenzo.

To generate new messages from the trained model, you can use the generate() method of the GPT2LMHeadModel class. This method generates a sequence of text given a prompt, using the probabilities learned during training.

In [89]:
# encode the starting sequence of tokens and move to GPU
input_ids = tokenizer.encode("<|startoftext|>", return_tensors="pt").to(model.device)

# generate new messages from the model
generated_text = model.generate(
    input_ids=input_ids, 
    max_length=50,
    do_sample=True,
    temperature=0.9,
    num_return_sequences=1
)

# decode the generated text
generated_messages = tokenizer.decode(generated_text[0], skip_special_tokens=True)

print("Generated messages:", generated_messages)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated messages: < | startoftext | > a me in questa ho mentre se e il mago e poi sono stati una ho trovato un mago per figlio tutti allora il mago io cos si ho mai e ho trovato un mago io cos per il mago e


In [97]:
model_path = "/content/gdrive/MyDrive/model"
tokenizer_path = "/content/gdrive/MyDrive/tokenizer"

In [104]:
import torch

tokenizer1 = BertTokenizer.from_pretrained(tokenizer_path)
model1 = GPT2LMHeadModel.from_pretrained(model_path)

# set the device to use for inference
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1.to(device)

# create the function to generate reply messages
def generate_text():

    # encode the user message
    input_ids = tokenizer1.encode("<|startoftext|>", return_tensors="pt").to(device)

    # generate the reply text
    generated_text = model1.generate(
        input_ids=input_ids,
        max_length=100,
        do_sample=True,
        temperature=0.7,
        num_return_sequences=1
    )
    
    # decode the generated text
    generated_message = tokenizer.decode(generated_text[0], skip_special_tokens=True)
    print(str(generated_message).replace("< | startoftext | > ", ""))

generate_text()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


a tutti i suoi il suo padre di soldi sicignano degli alburni ma il suo padre di soldi ma la bambina di modo e per la bambina di modo e per la bambina di modo e per la bambina di modo e per la bambina di modo e per la bambina di modo e per la bambina di modo e per la bambina di modo e per la bambina di modo e per la bambina di modo e per la bambina di modo e per la bambina
